# Pilot response model

The pilot response model aims to better capture unmanned aircraft response to advisories (either piloted by human or autonomously). When responding, the pilot executes the advisory for the total length of the period between decisions $T$. In the model, the advisory response in the current time step of $T$ is determined stochastically based on the new advisory via a Bernoulli process. As such, the delay until response follows a geometric distribution, and for some mean time until response $k$, the response probability at each step in the process is $T / (T + k)$.

Specifically,
* the pilot always response to a clear of conflict status "advisory"
* once the pilot responds, it will continue to respond for the duration of the advisory
* the average response delay for initial advisories is $5$ seconds (from ICAO recommended practices for responding to resolution advisories [1])
* when the pilot is not responding, the aircraft follows a white noise model.

[1] International Civil Aviation Organization, “Surveillance, radar and collision avoidance,” in International Standards and Recommended Practices, 4th, vol. IV, annex 10, 2007.

## Load modules

In [ ]:
push!(LOAD_PATH, "../dvi")

addprocs(int(CPU_CORES / 2))

using DiscreteValueIteration, GridInterpolations, PilotSCAs

mdp = SCA()

## Check size of MDP

In [ ]:
function getBytes(x)
   total = 0;
   fieldNames = typeof(x).names;
   if fieldNames == ()
      return sizeof(x);
   else
     for fieldName in fieldNames
        total += getBytes(getfield(x,fieldName));
     end
     return total;
   end
end

println("mdp of type ", typeof(mdp), " takes up ", getBytes(mdp) / 1000.0, " kB")

## Informal validation of state and action iterators and transition function

In [ ]:
nextStateIndices, probs = nextStates(mdp, 1, 15)
println("next state indices:\n", nextStateIndices, "\n")
println("probabilities:\n", probs, "\n")
println("probabilities sum to ", sum(probs))

## Parallel solution

In [ ]:
numProcs = int(CPU_CORES / 2)
solver = ParallelSolver(
    numProcs,
    maxIterations = 1000,
    tolerance = 1e-2,
    gaussSiedel = true,
    includeV = true,
    includeQ = true,
    includeA = true)

In [ ]:
policy = solve(solver, mdp, verbose = true)
println("")

In [ ]:
function sharray2array(sharray::SharedArray{Float64, 2})
    result = zeros(sharray.dims)
    for i = 1:sharray.dims[1]
        for j = 1:sharray.dims[2]
            result[i, j] = sharray[i, j]
        end # for j
    end # for i
    return result
end # function sharray2array

In [ ]:
using JLD

solQ = sharray2array(policy.Q')
save("../../data/pilot-alpha.jld", "solQ", solQ)

## Check against reference solution by visual inspection

In [ ]:
using PilotSCAViz

solQ = load("../../data/pilot-alpha.jld", "solQ")
println("")

### Parallel solver policy plot

In [ ]:
viz_pairwise_policy(d, solQ)